In [21]:
import numpy as np
import pickle
import os
from tkinter import *
import subprocess
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Flatten
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D
from keras import metrics

In [15]:
class MacOSFile(object):

    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        # print("reading total_bytes=%s" % n, flush=True)
        if n >= (1 << 31):
            buffer = bytearray(n)
            idx = 0
            while idx < n:
                batch_size = min(n - idx, 1 << 31 - 1)
                # print("reading bytes [%s,%s)..." % (idx, idx + batch_size), end="", flush=True)
                buffer[idx:idx + batch_size] = self.f.read(batch_size)
                # print("done.", flush=True)
                idx += batch_size
            return buffer
        return self.f.read(n)

    def write(self, buffer):
        n = len(buffer)
        print("writing total_bytes=%s..." % n, flush=True)
        idx = 0
        while idx < n:
            batch_size = min(n - idx, 1 << 31 - 1)
            print("writing bytes [%s, %s)... " % (idx, idx + batch_size), end="", flush=True)
            self.f.write(buffer[idx:idx + batch_size])
            print("done.", flush=True)
            idx += batch_size

def pickle_dump(obj, file_path):
    with open(file_path, "wb") as f:
        return pickle.dump(obj, MacOSFile(f), protocol=pickle.HIGHEST_PROTOCOL)


def pickle_load(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(MacOSFile(f))
    

In [23]:
print("Loading data.")
kjv_bible_data=pickle_load("../data/bible_data_20181123_update.pkl")
kjv_bible_mapping=pickle_load("../data/kjv_bible_mapping.pkl")
int2verse=pickle_load("../data/int2verse_mapping.pkl")
verse2int=pickle_load("../data/verse2int_mapping.pkl")
tf_idf_bible_fit=pickle_load('../data/tfidf_bible_fit.pkl')
tf_idf_bible_matrix=pickle_load('../data/tfidf_bible_matrix.pkl')
X=tf_idf_bible_matrix.todense()
y=np.array(list(map(lambda x:x[1][1], kjv_bible_mapping.items())))
def create_model(X,y):
    # Input layers
    print(X.shape)
    print(y.shape)
    model = Sequential()
    model.add(Dense(10000, input_shape=(X.shape[1],)))
    model.add(Dense(1000))
    model.add(Dense(100))
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['categorical_accuracy'])
    return model
def load_trained_model(weights_path, X, y):
    print("Loading model weights.")
    model = create_model(X,y)
    model.load_weights(weights_path)
    print("Load complete.")
    return model
model=load_trained_model("../data/weights-improvement-01-54.2576.hdf5",X,y)
print("Load complete.")


Loading data.
Loading model weights.
(31102, 12302)
(31102, 31102)
Load complete.
Load complete.


In [27]:
class Window(Frame):
    def __init__(self, master = None):
        Frame.__init__(self, master)
        self.master = master
        self.init_window()

    def init_window(self):
        self.master.title("Deep Connections (Beta 1.0.0)")
        self.pack(fill=BOTH, expand=1)

        self.cmd1 = StringVar()
        self.mEntry = Entry(self,textvariable=self.cmd1, bd=5, width=60)
        self.mEntry.pack()
        
        self.searchButton = Button(self, text="Search", command=self.OR_system_search)
        self.searchButton.pack()
        
        self.textbox = Text(height=100, width=1000)
        self.textbox.insert(END, "Results")
        self.textbox.pack()
        
        #installButton = Button(self, text="Install", command=self.system_install)
        #installButton.place(x=50, y=150)

    def OR_system_search(self):
        
        searchText = self.cmd1.get()
        tfidf_matrix = tf_idf_bible_fit.transform([searchText])
        x1=tfidf_matrix.todense()
        v=model.predict(x1)[0]
        indices = np.argsort(v)[-50:][::-1]
        topK=[]
        for index in indices:
            print
            #print(int2verse[index], kjv_bible_mapping[int2verse[index]][0], v)
            topK.append((int2verse[index], kjv_bible_mapping[int2verse[index]][0], v[index]))

        self.textbox.delete("1.0",END)

        for thing in topK:
            self.textbox.insert('1.0', thing[0] + "    " + thing[1] + "\n")
            print(thing[0], thing[2])
        
        
    def system_install(self):
        package = cmd1.get()
        install = "sudo pacman -S {} --noconfirm".format(package)
        subprocess.call([install], shell=True)

    def system_exit(self):
        exit()


root = Tk()
root.geometry("1240x680")
app = Window(root)

root.mainloop()

Matthew 24:43 0.003013836
John 8:44 0.002140234
John 8:52 0.002019988
Matthew 26:14 0.0019893053
John 6:71 0.00158769
John 13:2 0.0014763981
1 Corinthians 5:6 0.0013799866
Matthew 8:10 0.0012598963
John 6:70 0.0012529723
Matthew 23:7 0.0011666262
Matthew 26:45 0.0011126396
1 John 1:6 0.0009269326
Matthew 23:8 0.0009188495
Matthew 3:7 0.0008975613
Mark 7:17 0.00086148054
Mark 8:15 0.0008216137
Matthew 11:6 0.00080949493
1 Corinthians 9:22 0.00078658183
Mark 13:35 0.0007791046
Matthew 16:1 0.0007401772
Matthew 16:11 0.0007363339
Matthew 24:42 0.00071218365
1 Corinthians 11:19 0.0007076509
Matthew 17:20 0.0007062604
Acts 1:13 0.0007029945
1 Corinthians 8:9 0.0006861261
Romans 14:1 0.000683408
John 8:42 0.00064487767
Matthew 10:2 0.0006324039
Matthew 10:3 0.00062124
John 18:37 0.0006170024
Matthew 10:4 0.00061116
Romans 14:20 0.0006104307
John 10:20 0.00060874166
John 13:26 0.0006039849
Matthew 21:46 0.0005988983
Acts 23:6 0.0005964428
Luke 20:16 0.00059513794
1 John 4:14 0.00058817846
Act

In [28]:
!pip install py2app

    100% |████████████████████████████████| 604kB 3.3MB/s ta 0:00:011
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [29]:
import py2app